In [12]:
from operator import add
import pandas as pd
import itertools
from common import Data, Playoffs

In [13]:
df = pd.read_csv("data/2025.csv")
df["season"] = "2025"
teams = list(df["home"].unique())
Data.get_nwsl_standings(df)

,team,wins,draws,losses,goals_for,goals_against,goals_diff,points
1,KC,20,2,3,47.0,12.0,35.0,62
2,WAS,12,8,5,42.0,32.0,10.0,44
3,ORL,11,6,8,32.0,26.0,6.0,39
4,RGN,10,8,7,31.0,28.0,3.0,38
5,SD,10,7,8,40.0,32.0,8.0,37
6,POR,10,7,8,34.0,29.0,5.0,37
7,NJY,9,9,7,33.0,22.0,11.0,36
8,LOU,9,7,9,34.0,38.0,-4.0,34
9,NC,8,8,9,34.0,37.0,-3.0,32
10,HOU,8,6,11,27.0,37.0,-10.0,30


In [14]:
g = Playoffs.calc_games_matrix(df, teams)
p0 = Playoffs.calc_initial_points(df, teams)

for team in teams:
    print(
        team,
        "can finish:",
        Playoffs.calculate_highest_finish(team, teams, g, p0),
        "-",
        Playoffs.calculate_lowest_finish(team, teams, g, p0),
    )

HOU can finish: 9 - 11
ORL can finish: 3 - 7
KC can finish: 1 - 1
UTA can finish: 12 - 13
RGN can finish: 3 - 7
LOU can finish: 5 - 9
LA can finish: 10 - 11
POR can finish: 3 - 8
NC can finish: 8 - 10
WAS can finish: 2 - 2
SD can finish: 3 - 8
BFC can finish: 12 - 14
CHI can finish: 13 - 14
NJY can finish: 4 - 8


In [15]:
lookforward = 7
start = min(df.index[df["home_score"].isna()])
outcomes = list([(0, 0), (10, 0), (0, 10)])

In [16]:
options = []
for i in range(lookforward):
    for outcome in outcomes:
        options.append((i, outcome))


# Summarize the scenarios into something meaningful
def format_result(i, outcome):
    match = df.iloc[start + i]
    return f"{match['home']} {outcome[0]} - {match['away']} {outcome[1]}"


def summarize(df_results):
    scenarios = df_results.copy()

    # Check if all values are true
    if sum(scenarios["clinched"]) == len(scenarios.index):
        print("Clinched independent of results")
        return

    # Check single game criteria
    for (g1, o1) in options:
        subset = scenarios[scenarios[g1] == o1]
        if sum(subset["clinched"]) == len(subset.index):
            print("Clinched with", format_result(g1, o1))
            scenarios = scenarios[~(scenarios[g1] == o1)]

    # Check two game criteria
    for ((g1, o1), (g2, o2)) in itertools.combinations(options, 2):
        if g1 == g2:
            continue
        subset = scenarios[(scenarios[g1] == o1) & (scenarios[g2] == o2)]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print("Clinched with", format_result(g1, o1), "and", format_result(g2, o2))
            scenarios = scenarios[~((scenarios[g1] == o1) & (scenarios[g2] == o2))]

    # Check three game criteria
    for ((g1, o1), (g2, o2), (g3, o3)) in itertools.combinations(options, 3):
        if len(set([g1, g2, g3])) < 3:
            continue
        subset = scenarios[
            (scenarios[g1] == o1) & (scenarios[g2] == o2) & (scenarios[g3] == o3)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
            )
            scenarios = scenarios[
                ~((scenarios[g1] == o1) & (scenarios[g2] == o2) & (scenarios[g3] == o3))
            ]

    # Check four game criteria
    for ((g1, o1), (g2, o2), (g3, o3), (g4, o4)) in itertools.combinations(options, 4):
        if len(set([g1, g2, g3, g4])) < 4:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
                "and",
                format_result(g4, o4),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                )
            ]

    # Check five game criteria
    for ((g1, o1), (g2, o2), (g3, o3), (g4, o4), (g5, o5)) in itertools.combinations(
        options, 5
    ):
        if len(set([g1, g2, g3, g4, g5])) < 5:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
            & (scenarios[g5] == o5)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
                "and",
                format_result(g4, o4),
                "and",
                format_result(g5, o5),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                    & (scenarios[g5] == o5)
                )
            ]

    # Check six game criteria
    for (
        (g1, o1),
        (g2, o2),
        (g3, o3),
        (g4, o4),
        (g5, o5),
        (g6, o6),
    ) in itertools.combinations(options, 6):
        if len(set([g1, g2, g3, g4, g5, g6])) < 6:
            continue
        subset = scenarios[
            (scenarios[g1] == o1)
            & (scenarios[g2] == o2)
            & (scenarios[g3] == o3)
            & (scenarios[g4] == o4)
            & (scenarios[g5] == o5)
            & (scenarios[g6] == o6)
        ]
        if len(subset.index) == 0:
            continue
        if sum(subset["clinched"]) == len(subset.index):
            print(
                "Clinched with",
                format_result(g1, o1),
                "and",
                format_result(g2, o2),
                "and",
                format_result(g3, o3),
                "and",
                format_result(g4, o4),
                "and",
                format_result(g5, o5),
                "and",
                format_result(g6, o6),
            )
            scenarios = scenarios[
                ~(
                    (scenarios[g1] == o1)
                    & (scenarios[g2] == o2)
                    & (scenarios[g3] == o3)
                    & (scenarios[g4] == o4)
                    & (scenarios[g5] == o5)
                    & (scenarios[g6] == o6)
                )
            ]

In [17]:
# Calculate all scenarios for the following week and whether a specific team
# has clinched a playoff spot
def evaluate_scenarios(team, threshold):
    df_copy = df.copy()

    df_copy.loc[start : start + (lookforward - 1), "home_score"] = [0] * lookforward
    df_copy.loc[start : start + (lookforward - 1), "away_score"] = [0] * lookforward

    p0 = Playoffs.calc_initial_points(df, teams)
    g = Playoffs.calc_games_matrix(df_copy, teams)

    subset = df_copy.loc[start : start + (lookforward - 1)]

    def calculate_clinch(row):
        subset["home_score"] = list(row.str[0])
        subset["away_score"] = list(row.str[1])
        return (
            Playoffs.calculate_lowest_finish(
                team, teams, g, list(map(add, p0, Playoffs.calc_initial_points(subset, teams, True)))
            )
            <= threshold
        )

    scenarios = pd.DataFrame(itertools.product(outcomes, repeat=lookforward))
    scenarios["clinched"] = scenarios.apply(calculate_clinch, axis=1)
    return scenarios

In [18]:
# Calculate all scenarios for the following week and whether a specific team
# has been eliminated from the playoffs
def evaluate_elimination_scenarios(team, threshold):
    df_copy = df.copy()

    df_copy.loc[start : start + (lookforward - 1), "home_score"] = [0] * lookforward
    df_copy.loc[start : start + (lookforward - 1), "away_score"] = [0] * lookforward

    p0 = Playoffs.calc_initial_points(df, teams)
    g = Playoffs.calc_games_matrix(df_copy, teams)

    subset = df_copy.loc[start : start + (lookforward - 1)]

    def calculate_clinch(row):
        subset["home_score"] = list(row.str[0])
        subset["away_score"] = list(row.str[1])
        return (
            Playoffs.calculate_highest_finish(
                team, teams, g, list(map(add, p0, Playoffs.calc_initial_points(subset, teams, True)))
            )
            > threshold
        )

    scenarios = pd.DataFrame(itertools.product(outcomes, repeat=lookforward))
    scenarios["clinched"] = scenarios.apply(calculate_clinch, axis=1)
    return scenarios

## Clinching playoffs

Already clinched: KC, WAS, ORL, SD, NJY, POR, RGN

In [19]:
summarize(evaluate_scenarios("LOU", 8))

Clinched with NC 0 - NJY 0
Clinched with NC 0 - NJY 10
Clinched with LOU 10 - BFC 0


In [20]:
summarize(evaluate_scenarios("NC", 8))

Clinched with NC 10 - NJY 0 and LOU 0 - BFC 10


## Clinching hosting

Already clinched: KC, WAS

In [67]:
summarize(evaluate_scenarios("ORL", 4))

Clinched with RGN 0 - UTA 1 and WAS 0 - ORL 1 and SD 0 - CHI 0 and LA 0 - POR 0
Clinched with RGN 0 - UTA 1 and WAS 0 - ORL 1 and SD 0 - CHI 0 and LA 1 - POR 0
Clinched with RGN 0 - UTA 1 and WAS 0 - ORL 1 and SD 0 - CHI 1 and LA 0 - POR 0
Clinched with RGN 0 - UTA 1 and WAS 0 - ORL 1 and SD 0 - CHI 1 and LA 1 - POR 0


In [68]:
summarize(evaluate_scenarios("RGN", 4))

In [69]:
summarize(evaluate_scenarios("NJY", 4))

Clinched with RGN 0 - UTA 0 and WAS 0 - ORL 0 and SD 0 - CHI 1 and LA 1 - POR 0 and NJY 1 - LOU 0
Clinched with RGN 0 - UTA 0 and WAS 1 - ORL 0 and SD 0 - CHI 1 and LA 1 - POR 0 and NJY 1 - LOU 0
Clinched with RGN 0 - UTA 1 and WAS 0 - ORL 0 and SD 0 - CHI 1 and LA 1 - POR 0 and NJY 1 - LOU 0
Clinched with RGN 0 - UTA 1 and WAS 1 - ORL 0 and SD 0 - CHI 1 and LA 1 - POR 0 and NJY 1 - LOU 0


## Clinching shield

Already clinched: KC

In [13]:
summarize(evaluate_scenarios("KC", 1))

Clinched independent of results


## Eliminated from playoffs
Already eliminated: CHI, BAY, UTA, LA, HOU

In [70]:
summarize(evaluate_elimination_scenarios("LA", 8))

Clinched with LA 0 - POR 0
Clinched with LA 0 - POR 1
Clinched with NJY 0 - LOU 0
Clinched with NJY 0 - LOU 1


In [71]:
summarize(evaluate_elimination_scenarios("HOU", 8))

Clinched with HOU 0 - KC 0
Clinched with HOU 0 - KC 1
Clinched with NJY 0 - LOU 0
Clinched with NJY 0 - LOU 1


In [72]:
summarize(evaluate_elimination_scenarios("NC", 8))

Clinched with BFC 1 - NC 0
Clinched with BFC 0 - NC 0 and NJY 0 - LOU 0
Clinched with BFC 0 - NC 0 and NJY 0 - LOU 1


## Eliminated from hosting
Already eliminated: 

In [ ]:
summarize(evaluate_elimination_scenarios("NC", 4))

## Eliminated from shield
Already eliminated:

In [ ]:
summarize(evaluate_elimination_scenarios("KC", 1))

In [ ]:
summarize(evaluate_elimination_scenarios("NJY", 1))

In [ ]:
summarize(evaluate_elimination_scenarios("WAS", 1))